# Lösningsförslag till Övningsuppgift - Stopped Flow
Författare: Björn Dahlgren, 2016-04-04

Se övningsuppgiften för detaljer. Kortfattat ska vi bestämma hastighetskonstanten för följande reaktion:

$$
3A + B \rightarrow C
$$

Detta lösningsförslag är skrivet i Python, se bilaga laborationshandledning för liknande exempel skrivna för Matlab.

In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.linalg import lstsq
%matplotlib inline

Från uppgiftstexten vet vi att vi har data för 5 värden på [B]₀, 10 replikat för vardera, och 42 tidpunkter per mätserie. Vi läser in filerna in i variabeln ``data``:

In [ ]:
A0 = 1e-3
n_conc, n_rep, n_t = 5, 10, 42
conc_B = np.linspace(1/10, n_conc/10, n_conc)  # molar
data = np.empty((n_conc, n_rep, n_t, 2))
for i in range(n_conc):
    for j in range(n_rep):
        data[i, j, :, :] = np.loadtxt('data/0.%d/%d.dat' % (i+1, j+1))

Om vi antar pseudo tredje ordningens reaktion ([B]₀ » [A]₀) blir hastighetsuttrycket för reaktionen:

$$
\frac{dA}{dt} = -3kA^3
$$

integrering ger:
$$
A(t) = \frac{1}{\sqrt{\frac{1}{A_0^2} + 6kt}}
$$

genom Lambert-Beers lag får vi absorbansen ($y$):

$$
y(t) = \frac{\varepsilon l}{\sqrt{\frac{1}{A_0^2} + 6kt}}
$$

utifrån detta uttryck skapar vi en funktion för som vi kallar "``model``":

In [ ]:
def model(t, eps_l, k):
    return eps_l*(A0**-2 + 6*k*t)**-0.5

Nu genomför vi ickelinjärkurvanpassning av vår modellekvation på var och en av dataserierna. Vi formulerar även en relativ vikt baserad på hur väl passningen är. Valet av vikt är något godtyckligt och det kan vara bra att prova sig fram, ett vanligt val är Mean Square Deviation (MSD) & Mean Absolute Deviation (MAD). Vi använder MSD nedan:

In [ ]:
popt = np.empty((n_conc, n_rep, 2))  # y0, k
pcov = np.empty((n_conc, n_rep, 2, 2))
all_k = np.empty((n_conc, n_rep))
all_w = np.empty((n_conc, n_rep))
all_m = np.empty((n_conc, n_rep, n_t))
plt.figure(figsize=(16, 4))
for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    for j in range(n_rep):
        t = data[i, j, :, 0]
        y = data[i, j, :, 1]
        try:
            popt[i, j, :], pcov[i, j, :, :] = curve_fit(model, t, y, [y[0]/A0, 1])
        except:
            popt[i, j, :], pcov[i, j, :, :] = [0]*2, [[float('inf')]*2]*2
        plt.plot(t, y, 'k', alpha=0.4)
        all_k[i, j] = popt[i, j, 1]
        all_m[i, j, :] = m = model(t, *popt[i, j, :])
        # Val av vikt:
        #all_w[i, j] = 1/pcov[i, j, 0, 0]             # varians i epsilon*l
        #all_w[i, j] = 1/pcov[i, j, 1, 1]             # varians i k
        #all_w[i, j] = 1/np.average(np.square(y-m))    # MSD
        all_w[i, j] = 1/(np.average(np.abs(y-m)))    # MAD

for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
    ax.autoscale(tight=True)
    ax.set_ylabel('Absorbance')
    ax.set_xlabel('Time / s')
    for j in range(n_rep):
        red = 1 - all_w[i, j]/np.max(all_w[i, :])
        t = data[i, j, :, 0]
        plt.plot(t, all_m[i, j, :], c=(red, 0, 1.0))
plt.tight_layout()

Nu beräknar vi de viktade medelvärdena (se laborationshandledning för formel) för de olika värdena för [B]₀ baserat på respektive 10 replikat.

In [ ]:
k = np.empty(n_conc)
for i in range(n_conc):
    accum_k, accum_w = 0, 0
    for j in range(n_rep):
        w = all_w[i, j]
        accum_k += all_k[i, j]*w
        accum_w += w
    k[i] = accum_k/accum_w
k

Nu beräknar vi respektive viktade stickprovsvarians (se laborationshandledning för formel) för de nyligen beräknade viktade medelvärdena.

In [ ]:
s2k = np.empty_like(k)
for i in range(n_conc):
    accum_D, accum_w = 0, 0
    for j in range(n_rep):
        w = all_w[i, j]
        accum_D += w*(all_k[i, j] - k[i])**2
        accum_w += w
    s2k[i] = accum_D/((n_rep - 1)*accum_w)
s2k

Våra pseudo tredje ordningens hastighetskonstanter är linjärt beroende av [B]₀:

$$
k_f = [B]_0 k
$$

Därför beräknar vi $k_f$ genom [viktad linjär regression](https://en.wikipedia.org/wiki/Linear_least_squares_%28mathematics%29#Weighted_linear_least_squares):

In [ ]:
X = np.ones((n_conc, 2))
X[:, 1] = conc_B
W = np.diag(1/s2k)
XtW = (X.T).dot(W)
XtWX = XtW.dot(X)
beta = lstsq(XtWX, XtW.dot(k))[0]
M_beta = np.linalg.inv(XtWX)
beta, M_beta

In [ ]:
plt.errorbar(conc_B, k, 3*np.sqrt(s2k), c='k', ls='None', marker='.')
lbl = '$k_f = (%.2f \pm %.2g) M^{-3}s^{-1}$' % (beta[1], M_beta[1, 1]**0.5)
plt.plot(conc_B, beta[0] + beta[1]*conc_B, 'b', label=lbl)
plt.gca().set_xlim((0, conc_B[-1]*1.05))
plt.gca().set_ylim((0, k[-1]*1.1))
_ = plt.legend(loc='best')

Och därmed är uppgiften löst. Värdet på $k_f$ känns bekannt (eftersom våra indata är syntetiska stog det mig fritt att välja det "sanna" $k_f$).

Som "överkurs" kan vi ta fram ett antal statistiska värden med hjälp av paketet "statsmodels":

In [ ]:
import statsmodels.api as sm
sm.WLS(k, sm.add_constant(conc_B), weights=1/s2k).fit().summary()

Som ett alternativ till icke-linjär ekvationsanpassning skulle vi kunna ha linjäriserat vårt uttryck för absorbansen:

$$
\frac{1}{y^2(t)} = \frac{\frac{1}{A_0^2} + 6kt}{\varepsilon^2 l^2}
$$

därifrån kan vi lösa ut vårt pseudo tredje ordningens hastighetskonstant:

$$
\frac{1}{y^2(t)} = \frac{1}{\varepsilon^2 l^2 A_0^2} + \frac{6k}{\varepsilon^2 l^2}t \\
$$

notera att:

$$
\frac{1}{y^2(t)} \propto t
$$

vi inför därför en linjär funktion f(t) för regression:

$$
f(t) = p_0 + p_1 t
$$

$$
\begin{cases}
p_0 = \frac{1}{\varepsilon^2 l^2 A_0^2} \\
p_1 = \frac{6k}{\varepsilon^2 l^2}
\end{cases}
$$

$$
\begin{cases}
\varepsilon^2 l^2 = \frac{1}{p_0 A_0^2} \\
p_1 = \frac{6k}{\varepsilon^2 l^2}
\end{cases}
$$

$$
k = \frac{p_1}{6 p_0 A_0^2}
$$


In [ ]:
all_k = np.zeros((n_conc, n_rep))
plt.figure(figsize=(14, 4))
for i in range(n_conc):
    ax = plt.subplot(1, n_conc, i+1)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
    for j in range(n_rep):
        t = data[i, j, :, 0]
        f = data[i, j, :, 1]**-2
        plt.plot(t, f, c='k', alpha=0.3)
        p = np.polyfit(t, f, 1)
        all_k[i, j] = p[0]/(6*p[1]*1e-6)

In [ ]:
lin_k = np.mean(all_k, axis=1)
plt.plot(conc_B, lin_k, 's')
p = np.polyfit(conc_B, lin_k, 1)
plt.plot(conc_B, np.polyval(p, conc_B), label='k = %.2f' % p[0])
_ = plt.legend(loc='best')

Vi skulle kunna göra viktad regression även här men det lämnas till läsaren som uppgift.